In [2]:
import time
import os

import scanpy
import numpy as np
import scipy.sparse as sp

import torch
from torch import optim
from torch.utils.data import DataLoader

import models.modelsCNN as modelsCNN
import models.optimizer as optimizer

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import gc
from skimage import io
import scipy.stats
import utils.plot
import models.train_metaClf

from matplotlib.colors import LinearSegmentedColormap
from itertools import product
import matplotlib.patches as patches


/home/xzhang/anaconda3/envs/c2p/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '3'

resolution='5'
savedir_baseline='/data/xzhang/neuro/results/jointClustering/baseline/res'+resolution
with open(os.path.join(savedir_baseline,'pca_dapi_600'), 'rb') as output:
    pca_dapi=pickle.load(output)
with open(os.path.join(savedir_baseline,'pca_gfap_600'), 'rb') as output:
    pca_gfap=pickle.load(output)
with open(os.path.join(savedir_baseline,'pca_map2_600'), 'rb') as output:
    pca_map2=pickle.load(output)
with open(os.path.join(savedir_baseline,'pca_lmnb_600'), 'rb') as output:
    pca_lmnb=pickle.load(output)
with open(os.path.join(savedir_baseline,'modAll'), 'rb') as output:
    modAll=pickle.load(output)
with open(os.path.join(savedir_baseline,'silAll'), 'rb') as output:
    silAll=pickle.load(output)
with open(os.path.join(savedir_baseline,'moddapi'), 'rb') as output:
    moddapi=pickle.load(output)
with open(os.path.join(savedir_baseline,'sildapi'), 'rb') as output:
    sildapi=pickle.load(output)
with open(os.path.join(savedir_baseline,'modgfap'), 'rb') as output:
    modgfap=pickle.load(output)
with open(os.path.join(savedir_baseline,'silgfap'), 'rb') as output:
    silgfap=pickle.load(output)
with open(os.path.join(savedir_baseline,'modmap2'), 'rb') as output:
    modmap2=pickle.load(output)
with open(os.path.join(savedir_baseline,'silmap2'), 'rb') as output:
    silmap2=pickle.load(output)
with open(os.path.join(savedir_baseline,'modlmnb'), 'rb') as output:
    modlmnb=pickle.load(output)
with open(os.path.join(savedir_baseline,'sillmnb'), 'rb') as output:
    sillmnb=pickle.load(output)
with open(os.path.join(savedir_baseline,'leidenRes3_10samples'), 'rb') as output:
    leidenResAll=pickle.load(output)
    
savedir_processed='/data/xzhang/neuro/processed'
resDir='/data/xzhang/neuro/results/plots/cnnvaeexp0_segNucall'
saveDir=os.path.join(savedir_baseline,'cnnvaeexp0_segNucall')
if not os.path.exists(saveDir):
    os.mkdir(saveDir)


with open(os.path.join(resDir,'leiden_res'+resolution), 'rb') as output:
    finalPartition=pickle.load(output)
    
jointPartition=np.copy(finalPartition)
modFrac=0.3
modFrac_dapi=(moddapi/modAll>modFrac).astype(int)
modFrac_map2=(modmap2/modAll>modFrac).astype(int)
modFrac_gfap=(modgfap/modAll>modFrac).astype(int)
modFrac_lmnb=(modlmnb/modAll>modFrac).astype(int)
modFrac_sum=modFrac_dapi+modFrac_gfap+modFrac_lmnb+modFrac_map2

silAll_mean=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    silAll_mean[c]=np.mean(silAll[jointPartition==c])
    
sildapi_mean=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    sildapi_mean[c]=np.mean(sildapi[jointPartition==c])
    
silgfap_mean=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    silgfap_mean[c]=np.mean(silgfap[jointPartition==c])

silmap2_mean=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    silmap2_mean[c]=np.mean(silmap2[jointPartition==c])
sillmnb_mean=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    sillmnb_mean[c]=np.mean(sillmnb[jointPartition==c])
silAll_pos=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    silAll_pos[c]=np.sum(silAll[jointPartition==c]>0)/np.sum(jointPartition==c)
sildapi_pos=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    sildapi_pos[c]=np.sum(sildapi[jointPartition==c]>0)/np.sum(jointPartition==c)
silmap2_pos=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    silmap2_pos[c]=np.sum(silmap2[jointPartition==c]>0)/np.sum(jointPartition==c)
silgfap_pos=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    silgfap_pos[c]=np.sum(silgfap[jointPartition==c]>0)/np.sum(jointPartition==c)
sillmnb_pos=np.zeros(np.unique(jointPartition).size)
for c in np.unique(jointPartition):
    sillmnb_pos[c]=np.sum(sillmnb[jointPartition==c]>0)/np.sum(jointPartition==c)

silmean_frac=0.3
silmean_frac_neg=2
def comparesilmean(sildapi_mean,silAll_mean,silmean_frac,silmean_frac_neg):
    silmean_frac_dapi=np.zeros(silAll_mean.size)
    sildapi_mean_posIdx=np.arange(silAll_mean.size)[sildapi_mean>0]
    sildapi_mean_negIdx=np.arange(silAll_mean.size)[sildapi_mean<0]
#     print(sildapi_mean_negIdx[sildapi_mean[sildapi_mean<0]>(silAll_mean[sildapi_mean<0]*silmean_frac_neg)])
    silmean_frac_dapi[sildapi_mean_posIdx[sildapi_mean[sildapi_mean>0]>(silAll_mean[sildapi_mean>0]*silmean_frac)]]=1
    silmean_frac_dapi[sildapi_mean_negIdx[sildapi_mean[sildapi_mean<0]>(silAll_mean[sildapi_mean<0]*silmean_frac_neg)]]=1

    return silmean_frac_dapi

silmean_frac_dapi=comparesilmean(sildapi_mean,silAll_mean,silmean_frac,silmean_frac_neg)
silmean_frac_gfap=comparesilmean(silgfap_mean,silAll_mean,silmean_frac,silmean_frac_neg)
silmean_frac_map2=comparesilmean(silmap2_mean,silAll_mean,silmean_frac,silmean_frac_neg)
silmean_frac_lmnb=comparesilmean(sillmnb_mean,silAll_mean,silmean_frac,silmean_frac_neg)

silmean_frac_sum=silmean_frac_dapi+silmean_frac_gfap+silmean_frac_map2+silmean_frac_lmnb

silpos_frac=0.3
silpos_dapi=(sildapi_pos/silAll_pos>silpos_frac).astype(int)
silpos_gfap=(silgfap_pos/silAll_pos>silpos_frac).astype(int)
silpos_map2=(silmap2_pos/silAll_pos>silpos_frac).astype(int)
silpos_lmnb=(sillmnb_pos/silAll_pos>silpos_frac).astype(int)

silPos_sum=silpos_dapi+silpos_gfap+silpos_map2+silpos_lmnb


clusterIdx_pass=np.arange(np.unique(finalPartition).size)[(modFrac_sum>1)+(silmean_frac_sum>1)+(silPos_sum>1)]

nsamples=10
leidenJoint_overlap=np.zeros((np.unique(jointPartition).size,nsamples))
for cj in np.unique(jointPartition):
    for s in range(nsamples):
        _,counts=np.unique(leidenResAll[jointPartition==cj,s],return_counts=True)
        leidenJoint_overlap[cj,s]=np.max(counts)/np.sum(jointPartition==cj)

overlap_mean=np.mean(leidenJoint_overlap,axis=1)
overlap_std=np.std(leidenJoint_overlap,axis=1)

overlap_mean=np.mean(leidenJoint_overlap,axis=1)
overlap_std=np.std(leidenJoint_overlap,axis=1)

randomAll=np.zeros((leidenResAll.shape[0],nsamples)).astype(int)

for s in range(nsamples):
    randomS=np.copy(jointPartition)
    np.random.seed(s)
    np.random.shuffle(randomS)

    randomAll[:,s]=randomS
leidenJoint_overlap_random=np.zeros((np.unique(jointPartition).size,nsamples))
for cj in np.unique(jointPartition):
    for s in range(nsamples):
        _,counts=np.unique(randomAll[jointPartition==cj,s],return_counts=True)
        leidenJoint_overlap_random[cj,s]=np.max(counts)/np.sum(jointPartition==cj)

overlap_mean_random=np.mean(leidenJoint_overlap_random,axis=1)
overlap_std_random=np.std(leidenJoint_overlap_random,axis=1)

clusterIdx_robust=np.arange(np.unique(finalPartition).size)[(overlap_mean-overlap_std)>(overlap_mean_random+overlap_std_random)]
finalPartition_keep=np.intersect1d(clusterIdx_robust,clusterIdx_pass)


    
with open(os.path.join(savedir_processed,'meta_processed'), 'rb') as output:
    meta=pickle.load(output)

# annoUsed=np.array(['AAO', 'AAD', 'Duration', 'Gender','Clinical Diag','Brain Weight', 'Mutations',
#                    'APOE', 'Braak Tau', 'Thal Phase', 'CERAD', 'CAA', 'SVD','alpha-syn', 'TDP43'])
annoUsed=np.array(['Path Diag','Clinical Diag','Brain Weight', 'Mutations',
                   'APOE', 'Braak Tau', 'Thal Phase', 'CERAD', 'CAA', 'SVD','alpha-syn', 'TDP43',
                   'AAO', 'AAD', 'Duration', 'Gender'])
annoUsed_numeric=np.array(['AAO', 'AAD', 'Duration','Brain Weight','Braak Tau', 'Thal Phase', 'CERAD', 'CAA', 'SVD'])

meta=meta[annoUsed]
stains=np.array(['DAPI','GFAP','MAP2','LMNB'])

    
finalPartition_keepIdx=np.repeat(False, finalPartition.size)
for c in finalPartition_keep:
    finalPartition_keepIdx[finalPartition==c]=True
finalPartition_keep_input=np.copy(finalPartition)
finalPartition_keep_input[np.logical_not(finalPartition_keepIdx)]=-1



/home/xzhang/anaconda3/envs/c2p/lib/python3.10/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator PCA from version 0.24.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/tmp/ipykernel_1050035/1093437468.py:108: RuntimeWarning: invalid value encountered in divide
  silpos_dapi=(sildapi_pos/silAll_pos>silpos_frac).astype(int)
/tmp/ipykernel_1050035/1093437468.py:109: RuntimeWarning: divide by zero encountered in divide
  silpos_gfap=(silgfap_pos/silAll_pos>silpos_frac).astype(int)
/tmp/ipykernel_1050035/1093437468.py:110: RuntimeWarning: divide by zero encountered in divide
  silpos_map2=(silmap2_pos/silAll_pos>silpos_frac).astype(int)
/tmp/ipykernel_1050035/1093437468.py:111: RuntimeWarning: invalid value encountered in divide
  silpos_lmnb=(sillmnb_pos/silAll_pos>silpos

In [4]:
with open(os.path.join(savedir_processed,'allPatientID'), 'rb') as output:
    allPatientIDs=pickle.load(output)
meta=meta.loc[np.unique(allPatientIDs)]

with open(os.path.join(savedir_processed,'allImgNames'), 'rb') as output:
    allImgNames=pickle.load(output)
print(np.unique(allImgNames).size)
print(np.unique(allPatientIDs).size)

with open(os.path.join(savedir_processed,'allCoord'), 'rb') as output:
    allCoord=pickle.load(output)

64
50


In [5]:
allCoord.shape

(97530, 2)

In [6]:
def selectDenseRegions(x_coords,y_coords,topk,region_size):
    # Determine the bounding box of all coordinates
    x_min, x_max = np.min(x_coords), np.max(x_coords)
    y_min, y_max = np.min(y_coords), np.max(y_coords)

    # Generate candidate top-left corners of the regions
    x_candidates = np.arange(x_min, x_max - region_size + 1, step=50)  # Step size for efficiency
    y_candidates = np.arange(y_min, y_max - region_size + 1, step=50)

    # Dictionary to store cell counts in each region
    region_counts = {}


    for x, y in product(x_candidates, y_candidates):
        # Find points inside the 600x600 region
        mask = (x_coords >= x) & (x_coords < x + region_size) & \
               (y_coords >= y) & (y_coords < y + region_size)
        count = np.sum(mask)
        if count>0:
            region_counts[(x, y)] = count

    # Sort candidate regions by cell count (descending)
    sorted_regions = sorted(region_counts.items(), key=lambda x: x[1], reverse=True)

    # Select top 3 **disjoint** regions
    selected_regions = []
    for (x, y), count in sorted_regions:
        # Ensure new region does not overlap with already selected ones
        if all(abs(x - sx) >= region_size or abs(y - sy) >= region_size for sx, sy in selected_regions):
            selected_regions.append((x, y))
        if len(selected_regions) == topk:
            break
    return selected_regions

In [ ]:
regionSizes=[600,1300,2000]
topk=3
regionColors=['r','g','cornflowerblue']

savepath='/data/xzhang/neuro/results/jointClustering/baseline/res5/cnnvaeexp0_segNucall/clusterLocation_thresh8/'
dataDir='/data/xzhang/neuro'
proj2dpath='TIFS/2DMaxProj/2DMaxProj'
phenoLabels=['AD','Control','IPD','FTLD-TDPC','PSP']

if not os.path.exists(savepath):
    os.mkdir(savepath)
savepath_single=os.path.join(savepath,'single')
if not os.path.exists(savepath_single):
    os.mkdir(savepath_single)

cluster2plot=array([ 0.,  1.,  2.,  3.,  4.,  7., 11., 13., 14., 17., 18., 19., 20.,
       21., 28., 32., 34., 36., 39., 53., 62., 66., 74., 82., 84., 85.,
       94., 97., 99.])

imgSize=128
radius=int(imgSize/2)

minNucSize=300
laminThreshold=0.3 #by area
minmaxScale=(10,99.99)

black_to_magenta = LinearSegmentedColormap.from_list("black_to_magenta", ["black", "magenta"])#DAPI
black_to_cyan = LinearSegmentedColormap.from_list("black_to_magenta", ["black", "cyan"]) #map2
black_to_yellow = LinearSegmentedColormap.from_list("black_to_magenta", ["black", "orange"])#lmnb
black_to_green = LinearSegmentedColormap.from_list("black_to_magenta", ["black", "white"])#gfap
def apply_colormap(img, cmap):
    return cmap(img)[..., :3]
for p in phenoLabels:
    print(p)
    
    for s in os.listdir(os.path.join(dataDir,p,proj2dpath,'DAPI')):
        print(s)
        #load 2d dapi
        img2d=io.imread(os.path.join(dataDir,p,proj2dpath,'DAPI',s))
        img2d[img2d<np.percentile(img2d,minmaxScale[0])]=np.percentile(img2d,minmaxScale[0])
        img2d[img2d>np.percentile(img2d,minmaxScale[1])]=np.percentile(img2d,minmaxScale[1])
        img2d=(img2d-np.min(img2d))/(np.max(img2d)-np.min(img2d))
        #load 2d gfap
        img2d_gfap=io.imread(os.path.join(dataDir,p,proj2dpath,'GFAP',s))
        img2d_gfap[img2d_gfap<np.percentile(img2d_gfap,minmaxScale[0])]=np.percentile(img2d_gfap,minmaxScale[0])
        img2d_gfap[img2d_gfap>np.percentile(img2d_gfap,minmaxScale[1])]=np.percentile(img2d_gfap,minmaxScale[1])
        img2d_gfap=(img2d_gfap-np.min(img2d_gfap))/(np.max(img2d_gfap)-np.min(img2d_gfap))
        #load 2d map2
        img2d_map2=io.imread(os.path.join(dataDir,p,proj2dpath,'MAP2',s))
        img2d_map2[img2d_map2<np.percentile(img2d_map2,minmaxScale[0])]=np.percentile(img2d_map2,minmaxScale[0])
        img2d_map2[img2d_map2>np.percentile(img2d_map2,minmaxScale[1])]=np.percentile(img2d_map2,minmaxScale[1])
        img2d_map2=(img2d_map2-np.min(img2d_map2))/(np.max(img2d_map2)-np.min(img2d_map2))
        #load 2d lmnb
        img2d_lmnb=io.imread(os.path.join(dataDir,p,proj2dpath,'LMNB',s))
        img2d_lmnb[img2d_lmnb<np.percentile(img2d_lmnb,minmaxScale[0])]=np.percentile(img2d_lmnb,minmaxScale[0])
        img2d_lmnb[img2d_lmnb>np.percentile(img2d_lmnb,minmaxScale[1])]=np.percentile(img2d_lmnb,minmaxScale[1])
        img2d_lmnb=(img2d_lmnb-np.min(img2d_lmnb))/(np.max(img2d_lmnb)-np.min(img2d_lmnb))
        
        
        sname=s.split('.')[0]
#         currImgName=np.repeat(sname,imgCount)
        currPatientID=sname.split('-')[0]+'-'+sname.split('-')[1]
        
        imgIdx=allImgNames==sname
        
        denseRegions={}
        for c in cluster2plot:
            if not os.path.exists(os.path.join(savepath_single,str(c))):
                os.mkdir(os.path.join(savepath_single,str(c)))
                
            cidx=finalPartition_keep_input==c
            
            currCoord=np.logical_and(imgIdx,cidx)
    
            fig, ax = plt.subplots()

            img_dapi_rgb = apply_colormap(img2d, black_to_magenta)
            img_gfap_rgb = apply_colormap(img2d_gfap, black_to_green)
            img_map2_rgb = apply_colormap(img2d_map2, black_to_cyan)
            img_lmnb_rgb = apply_colormap(img2d_lmnb, black_to_yellow)

            # Merge all channels (ensuring no saturation)
            composite = img_dapi_rgb + img_gfap_rgb + img_map2_rgb + img_lmnb_rgb
#             composite=composite/np.max(composite)
#             composite = img_dapi_rgb + img_gfap_rgb 
            ax.imshow(composite)
            ax.scatter(allCoord[currCoord,1], allCoord[currCoord,0], s=8, c='gold', edgecolors='black',linewidths=0.3)  # Cluster points
            
            denseRegions[c]={}
            for srIdx in range(len(regionSizes)):
                if np.sum(currCoord)>0:
                    denseRegions[c][srIdx]=selectDenseRegions(allCoord[currCoord, 0], allCoord[currCoord, 1],topk,regionSizes[srIdx])
                else:
                    denseRegions[c][srIdx]=[]
#             plt.show()
            plt.savefig(os.path.join(savepath_single,str(c),p+'_'+sname+'_fullImg.jpg'),dpi=2000)
            plt.close()
        
        for clusterIdx in range(len(cluster2plot)):
            fig, ax = plt.subplots()
            ax.imshow(composite)
            
            c=cluster2plot[clusterIdx]

            cidx=finalPartition_keep_input==c
            
            currCoord=np.logical_and(imgIdx,cidx)

            ax.scatter(allCoord[currCoord,1], allCoord[currCoord,0], s=8, c='gold', edgecolors='black',linewidths=1)  # Cluster points
            for srIdx in range(len(regionSizes)):
                selected_regions=denseRegions[c][srIdx]
                for ridx in range(len(selected_regions)):
                    rect = patches.Rectangle((selected_regions[ridx][1], selected_regions[ridx][0]),regionSizes[srIdx],regionSizes[srIdx], linewidth=2, edgecolor=regionColors[srIdx], facecolor='none')
                    ax.add_patch(rect)
            
            plt.savefig(os.path.join(savepath_single,str(c),p+'_'+sname+'_sampledRegions_cluster'+str(c)+'.jpg'),dpi=2000)
            plt.close()
        
        for clusterIdx in range(len(cluster2plot)):
            c=cluster2plot[clusterIdx]

            cidx=finalPartition_keep_input==c

            currCoord=np.logical_and(imgIdx,cidx)

            for srIdx in range(len(regionSizes)):
                selected_regions=denseRegions[c][srIdx]
                for ridx in range(len(selected_regions)):
                    fig, ax = plt.subplots()
                    ax.imshow(composite[int(selected_regions[ridx][0]):int(selected_regions[ridx][0])+regionSizes[srIdx], int(selected_regions[ridx][1]):int(selected_regions[ridx][1])+regionSizes[srIdx]])

                    recCoord=allCoord[currCoord]
                    recCoord=recCoord[np.logical_and(recCoord[:,0]>selected_regions[ridx][0],recCoord[:,0]<selected_regions[ridx][0]+regionSizes[srIdx])]
                    recCoord=recCoord[np.logical_and(recCoord[:,1]>selected_regions[ridx][1],recCoord[:,1]<selected_regions[ridx][1]+regionSizes[srIdx])]
                    ax.scatter(recCoord[:,1]-int(selected_regions[ridx][1]), recCoord[:,0]-int(selected_regions[ridx][0]), s=20, c='gold', edgecolors='black',linewidths=1)  # Cluster points
            
                    plt.savefig(os.path.join(savepath_single,str(c),p+'_'+sname+'_sampledRegions_cluster'+str(c)+'_size'+str(regionSizes[srIdx])+'_region'+str(ridx)+'.jpg'),dpi=2000)
                    plt.close()
                    
                    fig, ax = plt.subplots()
                    ax.imshow(composite[int(selected_regions[ridx][0]):int(selected_regions[ridx][0])+regionSizes[srIdx], int(selected_regions[ridx][1]):int(selected_regions[ridx][1])+regionSizes[srIdx]])
            
                    plt.savefig(os.path.join(savepath_single,str(c),p+'_'+sname+'_sampledRegions_cluster'+str(c)+'_size'+str(regionSizes[srIdx])+'_region'+str(ridx)+'_nodots.jpg'),dpi=2000)
                    plt.close()


In [5]:
with open(os.path.join(savedir_baseline,'clusterPlots_bymeta_igFilter_thresh8','clusterNeigh'), 'rb') as output:
    clusterNeigh=pickle.load(output)

In [6]:
#two main neighborhood patterns
neigh1_cluster=np.array([ 0.,  3., 13., 14., 18., 36., 39., 62., 66., 74., 82., 94.])
neigh2_cluster=np.array([ 1.,  4., 11., 17., 21., 28., 32., 53., 97., 99.])

In [7]:
neigh1_cluster

array([ 0.,  3., 13., 14., 18., 36., 39., 62., 66., 74., 82., 94.])

In [8]:
neigh2_cluster

array([ 1.,  4., 11., 17., 21., 28., 32., 53., 97., 99.])

In [ ]:
cluster2plot=[neigh1_cluster,neigh2_cluster]
regionSizes=[600,1300,2000]
topk=3
regionColors=['r','g','cornflowerblue']
savepath='/data/xzhang/neuro/results/jointClustering/baseline/res5/cnnvaeexp0_segNucall/clusterLocation_thresh8/'
dataDir='/data/xzhang/neuro'
proj2dpath='TIFS/2DMaxProj/2DMaxProj'
phenoLabels=['Control','AD','IPD','FTLD-TDPC','PSP']
plotMeta=['Path Diag','APOE','TDP43']
colors = ['cornflowerblue', 'gold', 'chocolate']

if not os.path.exists(savepath):
    os.mkdir(savepath)
savepath_neigh=os.path.join(savepath,'neigh')
if not os.path.exists(savepath_neigh):
    os.mkdir(savepath_neigh)

imgSize=128
radius=int(imgSize/2)

minNucSize=300
laminThreshold=0.3 #by area
minmaxScale=(10,99.99)

black_to_magenta = LinearSegmentedColormap.from_list("black_to_magenta", ["black", "magenta"])#DAPI
black_to_cyan = LinearSegmentedColormap.from_list("black_to_magenta", ["black", "cyan"]) #map2
black_to_yellow = LinearSegmentedColormap.from_list("black_to_magenta", ["black", "orange"])#lmnb
black_to_green = LinearSegmentedColormap.from_list("black_to_magenta", ["black", "white"])#gfap
def apply_colormap(img, cmap):
    return cmap(img)[..., :3]
for p in phenoLabels:
    print(p)
    
    for s in os.listdir(os.path.join(dataDir,p,proj2dpath,'DAPI')):
        print(s)
        #load 2d dapi
        img2d=io.imread(os.path.join(dataDir,p,proj2dpath,'DAPI',s))
        img2d[img2d<np.percentile(img2d,minmaxScale[0])]=np.percentile(img2d,minmaxScale[0])
        img2d[img2d>np.percentile(img2d,minmaxScale[1])]=np.percentile(img2d,minmaxScale[1])
        img2d=(img2d-np.min(img2d))/(np.max(img2d)-np.min(img2d))
        #load 2d gfap
        img2d_gfap=io.imread(os.path.join(dataDir,p,proj2dpath,'GFAP',s))
        img2d_gfap[img2d_gfap<np.percentile(img2d_gfap,minmaxScale[0])]=np.percentile(img2d_gfap,minmaxScale[0])
        img2d_gfap[img2d_gfap>np.percentile(img2d_gfap,minmaxScale[1])]=np.percentile(img2d_gfap,minmaxScale[1])
        img2d_gfap=(img2d_gfap-np.min(img2d_gfap))/(np.max(img2d_gfap)-np.min(img2d_gfap))
        #load 2d map2
        img2d_map2=io.imread(os.path.join(dataDir,p,proj2dpath,'MAP2',s))
        img2d_map2[img2d_map2<np.percentile(img2d_map2,minmaxScale[0])]=np.percentile(img2d_map2,minmaxScale[0])
        img2d_map2[img2d_map2>np.percentile(img2d_map2,minmaxScale[1])]=np.percentile(img2d_map2,minmaxScale[1])
        img2d_map2=(img2d_map2-np.min(img2d_map2))/(np.max(img2d_map2)-np.min(img2d_map2))
        #load 2d lmnb
        img2d_lmnb=io.imread(os.path.join(dataDir,p,proj2dpath,'LMNB',s))
        img2d_lmnb[img2d_lmnb<np.percentile(img2d_lmnb,minmaxScale[0])]=np.percentile(img2d_lmnb,minmaxScale[0])
        img2d_lmnb[img2d_lmnb>np.percentile(img2d_lmnb,minmaxScale[1])]=np.percentile(img2d_lmnb,minmaxScale[1])
        img2d_lmnb=(img2d_lmnb-np.min(img2d_lmnb))/(np.max(img2d_lmnb)-np.min(img2d_lmnb))
        
        
        sname=s.split('.')[0]
        
#         currImgName=np.repeat(sname,imgCount)
        currPatientID=sname.split('-')[0]+'-'+sname.split('-')[1]
        
        imgIdx=allImgNames==sname
        

        savepath_neigh_curr=os.path.join(savepath_neigh,p+'_'+sname)
        if os.path.exists(savepath_neigh_curr):
            continue

        fig, ax = plt.subplots()

        img_dapi_rgb = apply_colormap(img2d, black_to_magenta)
        img_gfap_rgb = apply_colormap(img2d_gfap, black_to_green)
        img_map2_rgb = apply_colormap(img2d_map2, black_to_cyan)
        img_lmnb_rgb = apply_colormap(img2d_lmnb, black_to_yellow)

        composite = img_dapi_rgb + img_gfap_rgb + img_map2_rgb + img_lmnb_rgb
        ax.imshow(composite)

        denseRegions={}
        for clusterIdx in range(len(cluster2plot)):
            c=cluster2plot[clusterIdx]

            cidx=np.repeat(False,finalPartition_keep_input.size)
            for ci in c:
                cidx[finalPartition_keep_input==ci]=True

            currCoord=np.logical_and(imgIdx,cidx)


            ax.scatter(allCoord[currCoord,1], allCoord[currCoord,0], s=8, c=colors[clusterIdx], edgecolors='black',linewidths=1)  # Cluster points

            denseRegions[clusterIdx]={}
            for srIdx in range(len(regionSizes)):
                denseRegions[clusterIdx][srIdx]=selectDenseRegions(allCoord[currCoord, 0], allCoord[currCoord, 1],topk,regionSizes[srIdx])
                
#             plt.show()
        plt.savefig(savepath_neigh_curr+'_fullImg.jpg',dpi=2000)
        plt.close()
        
        for clusterIdx in range(len(cluster2plot)):
            fig, ax = plt.subplots()
            ax.imshow(composite)
            
            c=cluster2plot[clusterIdx]

            cidx=np.repeat(False,finalPartition_keep_input.size)
            for ci in c:
                cidx[finalPartition_keep_input==ci]=True

            currCoord=np.logical_and(imgIdx,cidx)

            ax.scatter(allCoord[currCoord,1], allCoord[currCoord,0], s=8, c=colors[clusterIdx], edgecolors='black',linewidths=1)  # Cluster points
            for srIdx in range(len(regionSizes)):
                selected_regions=denseRegions[clusterIdx][srIdx]
                for ridx in range(len(selected_regions)):
                    rect = patches.Rectangle((selected_regions[ridx][1], selected_regions[ridx][0]),regionSizes[srIdx],regionSizes[srIdx], linewidth=2, edgecolor=regionColors[srIdx], facecolor='none')
                    ax.add_patch(rect)
            
            plt.savefig(savepath_neigh_curr+'_sampledRegions_cluster'+str(clusterIdx)+'.jpg',dpi=2000)
            plt.close()
        
        for clusterIdx in range(len(cluster2plot)):
            c=cluster2plot[clusterIdx]

            cidx=np.repeat(False,finalPartition_keep_input.size)
            for ci in c:
                cidx[finalPartition_keep_input==ci]=True

            currCoord=np.logical_and(imgIdx,cidx)

            for srIdx in range(len(regionSizes)):
                selected_regions=denseRegions[clusterIdx][srIdx]
                for ridx in range(len(selected_regions)):
                    fig, ax = plt.subplots()
                    ax.imshow(composite[int(selected_regions[ridx][0]):int(selected_regions[ridx][0])+regionSizes[srIdx], int(selected_regions[ridx][1]):int(selected_regions[ridx][1])+regionSizes[srIdx]])

                    recCoord=allCoord[currCoord]
                    recCoord=recCoord[np.logical_and(recCoord[:,0]>selected_regions[ridx][0],recCoord[:,0]<selected_regions[ridx][0]+regionSizes[srIdx])]
                    recCoord=recCoord[np.logical_and(recCoord[:,1]>selected_regions[ridx][1],recCoord[:,1]<selected_regions[ridx][1]+regionSizes[srIdx])]
                    ax.scatter(recCoord[:,1]-int(selected_regions[ridx][1]), recCoord[:,0]-int(selected_regions[ridx][0]), s=20, c=colors[clusterIdx], edgecolors='black',linewidths=1)  # Cluster points
            
                    plt.savefig(savepath_neigh_curr+'_sampledRegions_cluster'+str(clusterIdx)+'_size'+str(regionSizes[srIdx])+'_region'+str(ridx)+'.jpg',dpi=2000)
                    plt.close()
                    
                    fig, ax = plt.subplots()
                    ax.imshow(composite[int(selected_regions[ridx][0]):int(selected_regions[ridx][0])+regionSizes[srIdx], int(selected_regions[ridx][1]):int(selected_regions[ridx][1])+regionSizes[srIdx]])
            
                    plt.savefig(savepath_neigh_curr+'_sampledRegions_cluster'+str(clusterIdx)+'_size'+str(regionSizes[srIdx])+'_region'+str(ridx)+'_nodots.jpg',dpi=2000)
                    plt.close()
                    
                    